In [1]:
import pandas as pd

In [3]:
pd.read_csv("./data/cleaned_test_set_VU_DM.csv").columns

/tmp/ipykernel_106028/3443471649.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("./data/cleaned_test_set_VU_DM.csv").columns


Index(['visitor_location_country_id', 'prop_country_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'random_bool', 'weekday', 'part_of_day',
       'last_minute', 'season_booking'],
      dtype='object')

In [13]:
df = pd.read_csv("./pred.csv")

In [21]:
pd.read_csv("./data/cleaned_test_set_VU_DM.csv").isna().any()

/tmp/ipykernel_106028/660993057.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("./data/cleaned_test_set_VU_DM.csv").isna().any()


srch_id                        False
visitor_location_country_id    False
prop_country_id                False
prop_id                        False
prop_starrating                False
prop_review_score              False
prop_brand_bool                False
prop_location_score1           False
prop_log_historical_price      False
price_usd                      False
promotion_flag                 False
srch_destination_id            False
srch_length_of_stay            False
srch_booking_window            False
srch_adults_count              False
srch_children_count            False
srch_room_count                False
srch_saturday_night_bool       False
random_bool                    False
weekday                        False
part_of_day                    False
last_minute                    False
season_booking                 False
dtype: bool

In [27]:
from preprocessing import season_booking
path="./data/test_set_VU_DM.csv"
df = pd.read_csv(path)
df.shape

(4959183, 50)

In [28]:
if "train" in path:
    df = df.drop(['position', 'gross_bookings_usd'], axis=1)
df.shape

(4959183, 50)

In [29]:
df['date_time'] = pd.to_datetime(df['date_time'])
print(df.shape)
df['weekday'] = df['date_time'].dt.dayofweek
print(df.shape)
df['part_of_day'] = pd.cut(df['date_time'].dt.hour,[0,6,12,18,24],labels=[0,1,2,3],include_lowest=True) # 'night','morning','afternoon','evening' , 0-4
print(df.shape)
df['last_minute'] = [1 if x <= 14 else 0 for x in df['srch_booking_window']]
print(df.shape)
df['date_booking'] = df['date_time'] + pd.to_timedelta(df['srch_booking_window'], unit='d')
print(df.shape)
df['date_booking'] = df['date_time'] + pd.to_timedelta(df['srch_booking_window'], unit='d')
print(df.shape)
df['season_booking'] = df.apply(season_booking, axis=1)
print(df.shape)
df.drop(['date_booking'], axis=1, inplace=True)
print(df.shape)
df['prop_review_score'] = df['prop_review_score'].fillna('bfill')
print(df.shape)
df['comp_rate'] = [1 if all(x == 1 for x in row) else 0 if all(x == 0 for x in row) else -1 for row in df[['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate']].values]
print(df.shape)
columns = ['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate', 'comp6_rate', 'comp7_rate', 'comp8_rate']
print(df.shape)
for column in columns:
    df[column] = df[column] + 1
    df['comp_rate'] = df.groupby('prop_id')[column].transform('mean')
print(df.shape)
columns2 = ['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']
for column in columns2:
    df[column] = df[column] + 1
    df['comp_inv'] = df.groupby('prop_id')[column].transform('mean')
print(df.shape)
columns2 = ['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']
for column in columns2:
    df['comp_rate_percent_diff'] = df.groupby('prop_id')[column].transform('mean')
print(df.shape)
df = df.dropna(axis=1, how="any")
print(df.shape)
df = df.drop(['date_time', 'site_id'],axis=1)
print(df.shape)
if "train" in path:
    df = df.dropna(axis=0)
print(df.shape)

(4959183, 50)
(4959183, 51)
(4959183, 52)
(4959183, 53)
(4959183, 54)
(4959183, 54)
(4959183, 55)
(4959183, 54)
(4959183, 54)
(4959183, 55)
(4959183, 55)
(4959183, 55)
(4959183, 56)
(4959183, 57)
(4959183, 25)
(4959183, 23)
(4959183, 23)


In [31]:
import os
df.to_csv(f"./data/cleaned_{os.path.basename(path)}", index=False)


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
df = pd.read_csv("./data/cleaned_training_set_VU_DM.csv").drop(["prop_review_score"],axis=1)
df = (df - df.min()) / (df.max() - df.min()) #Normalize the data
columns = df.drop('rating', axis=1).columns
target = np.expand_dims(df['rating'].to_numpy(),-1)
features = df.drop('rating', axis=1).to_numpy()
data = train_test_split(features, target, test_size=0.1) #10% for validation
x_train, x_val, y_train, y_val = data
x_train = np.append(x_train,y_train, axis=1)

In [3]:
U, S, V = np.linalg.svd(x_train, full_matrices=False)

In [13]:
k = 3
U_k = U[:, :k]
S_k = np.diag(S[:k])
V_k = V[:k, :]

In [15]:
# Predict the missing column for the test matrix
missing_column = x_val @ V_k.T[-1]  # Adjust -1 to select the appropriate column index from V_k.T

# Print the predicted missing column
print("Predicted Missing Column:")
print(missing_column)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 20)

In [20]:
x_val.shape, np.expand_dims(V_k[:,-1],-1).shape

((495835, 20), (3, 1))

In [25]:
V.shape, U.shape, S.shape

((21, 21), (4462512, 21), (21,))

In [23]:
V_k.shape, U_k.shape, S_k.shape

((3, 21), (4462512, 3), (3, 3))

In [1]:
import pandas as pd

In [14]:
df = pd.read_csv("./data/cleaned_training_set_VU_DM.csv")['srch_id']
df.unique()

array([     1,      4,      6, ..., 332782, 332784, 332785])

In [19]:
df.min(), df.max(), df2['srch_id'].min(), df2['srch_id'].max()

(1, 332785, 332786, 665572)

In [15]:
df2 = pd.read_csv("./data/cleaned_test_set_VU_DM.csv")

In [18]:
df2['srch_id'] += 332785

In [20]:
df2.to_csv("./data/testing_ffm.csv")

In [22]:
df2.columns

Index(['srch_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'random_bool', 'weekday',
       'part_of_day', 'last_minute', 'season_booking'],
      dtype='object')

In [25]:
df.isin(['bfill']).any()

False